### Basic PRM Algorithm

1. construct a roadmap in a probabilistic way (for a given workspace)
  - roadmap is represented by an undirected graph $G= (V,E)$.
  - nodes in $G$ are robot configurations sampled randomly from a uniform  distribution over $Q_{free}$, the free configuration space.
  - edges in $E$ correspond to collision-free paths; an edge $(q1, q2)$ is such a path connecting configurations $q_1$ and $q_2$.
  - each edge is a local path generated by a local planner, $\Delta$: a simple local planner connects two nodes by a straight line in $Q_{free}$, if such a line exists.

2. query roadmap to solve for a path between a start and a goal configuration ($q_{start}$ and $q_{goal}$, respectively)
  - connect $q_{start}$ and $q_{goal}$ to two nodes $q'$ and $q''$, respectively, in $V$, if possible
  - then search the graph for a sequence of edges in $E$ connecting $q'$ to $q''$
  - then recompute local paths and concatenate them from $q_{start}$ to $q_{goal}$

In [ ]:
# Algorithm 1: roadmap construction algorithm (pseudocode)

# Inputs:
# n: number of nodes to put in the roadmap
# k: number of closest neighbours to examine for each configuration
#
# Output:
# roadmap G = (V,E)

# V = {}
# E = {}
#
# while len(V) < n:
#
#   while True:
#     q = sample_configuration(Q, uniform)
#
#     if is_collision_free(q):
#       V.append(q)
#       break
#
# for q in V:
#   q_neighbourhood = get_neighbours(q, k, distance_metric, Q)
#
#     for q_neighbour in q_neighbourhood:
#       if (q, q_neighbour) not in E and \
#           local_path_exists(q, q_neighbour, local_planner):
#         E.append((q, q_neighbour))


In [ ]:
# Implementation of Algorithm 1

# Assumptions:
# Q is the closed bounded 2D Euclidean square from [0,0] to [x,y]
# each obstacle is a closed disk in Q defined by centre coordinates and a radius
# local_planner finds a straight line path between points or returns None

from decimal import Decimal
from typing import Tuple, List

# define vector class
class Vector2D:
  """
    This class represents a 2D vector.
    Args:
      x (float): The x-coordinate of the vector.
      y (float): The y-coordinate of the vector.

    Attributes:
      x (float): The x-coordinate of the vector.
      y (float): The y-coordinate of the vector.

    Methods:
      norm(self, vector): Calculate the magnitude of the vector.
      __sub__(self, other): Overload the minus operator to perform vector subtraction.
      __add__(self, other): Overload the plus operator to perform vector addition.
      distance(self, other): Calculate the distance between two vectors.
      displacement(self, other): Aliases vector subtraction.
      unit_vector(self): Calculate the unit vector of the vector.

    Sample code:
      ```python
        # Create two vector instances
        v1 = Vector(3, 4)
        v2 = Vector(1, 2)

        # Calculate the norm of v1
        print(Vector.norm(v1))  # Output: 5.0

        # Calculate the displacement between v1 and v2
        displacement = v1 - v2
        print(displacement.x, displacement.y)  # Output: (2, 2)

        # Calculate the distance between v1 and v2
        print(v1.distance(v2))  # Output: 2.8284271247461903

        # Calculate the unit vector of v1
        unit_v1 = v1.unit_vector()
        print(unit_v1.x, unit_v1.y)  # Output: (0.6, 0.8)
      ```
  """

  def __init__(self, x, y):
    self.x = x
    self.y = y

  @classmethod
  def from_configuration2D(cls, config):
    """ create Vector2D object from a Configuration2D object """

    return cls(config.x, config.y)

  @classmethod
  def get_displacement_vector(cls, start_vector, end_vector):
    """ Calculate displacement vector that points from self to end """

    return end_vector - start_vector

  # define static norm method
  @staticmethod
  def norm(vector):
    """ Calculate the magnitude(length) of the vector """

    return (vector.x ** 2 + vector.y ** 2) ** 0.5

  def __sub__(self, other):
    """ Overload the minus operator to perform vector subtraction """

    return Vector2D(self.x - other.x, self.y - other.y)

  def __add__(self, other):
    """ Overload the plus operator to perform vector addition """

    return Vector2D(self.x + other.x, self.y + other.y)

  def __truediv__(self, scalar):
    """ Overload division operator to perform scalar division """

    # TODO: catch divide by zero error
    return self * (1 / scalar)

  def __mul__(self, scalar):
    """ Overload the multiply operator to perform scalar multiplication """

    return Vector2D(self.x * scalar, self.y * scalar)

  def distance(self, other):
    """ Calculate the scalar distance between two vectors """

    return Vector2D.norm(other - self)

  def unit_vector(self):
    """ Calculate the unit vector """

    mag = Vector2D.norm(self)
    return self / mag

# define configuration class with immutable x,y coordinates
class Configuration2D:
  """ Class representing a 2D configuration. """

  def __init__(self, x, y):
    """ Initialize the 2D configuration with x and y coordinates """

    self.x = x
    self.y = y

  @classmethod
  def from_vector(cls, vector):
    """ create Configuration2D object from a Vector2D object """

    return cls(vector.x, vector.y)

  # TODO: Implement immutable coordinates and update docstring

  def __eq__(self, other):
    return self.x == other.x and self.y == other.y

  def __str__(self):
    return f'Configuration2D({self.x},{self.y})'

# define node class
class Node:
  def __init__(self, configuration):
    self.configuration = configuration

  def __repr__(self):
    return f"Node({self.configuration})"

# define obstacle class
class DiskObstacle2D:
  """
    This class represents a disk-shaped obstacle.

    Args:
      centre (tuple): The (x, y) coordinates of the center of the disk.
      radius (float): The radius of the disk.

    Attributes:
      centre (tuple): The (x, y) coordinates of the center of the disk.
      radius (float): The radius of the disk.
  """

  def __init__(self, centre, radius):
    self.centre = centre
    self.radius = radius

  def colliding_with(self, configuration):
    # check if the configuration is within the obstacle
    return self.contains(configuration)

  def contains(self, configuration):
    from math import hypot
    # calculate the distance between the configuration and the obstacle centre
    distance = hypot(configuration.x - self.centre[0],
                     configuration.y - self.centre[1])

    return distance <= self.radius

# define bounds class
class RectangularBounds2D:

  def __init(self,
             x_bounds: Tuple[float, float],
             y_bounds: Tuple[float, float]):
    # set boundary extremes
    self.x_min = Decimal(min(x_bounds))
    self.x_max = Decimal(max(x_bounds))
    self.y_min = Decimal(min(y_bounds))
    self.y_max = Decimal(max(y_bounds))

    self.x_bounds = (self.x_min, self.x_max)
    self.y_bounds = (self.y_min, self.y_max)

  def is_within_bounds(self, configuration):
    # check if the configuration is within bounds
    in_x_bounds = configuration.x >= self.x_min and configuration.x <= self.x_max
    in_y_bounds = configuration.y >= self.y_min and configuration.y <= self.y_max

    if in_x_bounds and in_y_bounds:
      return True

    return False

# define configuration space class
class ConfigurationSpace2D:
  def __init__(self, bounds, obstacle_list):
    # initialise configuration space with bounds, obstacle list, metric
    self.bounds = bounds
    self.obstacle_list = obstacle_list

  def is_collision_free(self, configuration):
    # check if configuration is in bounds
    if not self.bounds.is_within_bounds(configuration):
      return False

    # check if the given configuration is in collision with any obstacles
    for obstacle in self.obstacle_list:
      if obstacle.colliding_with(configuration):
        return False

    return True


  def metric(self, start, goal):
    # Implement distance metric here
    pass

# define local planner class: construct obstacle-free edge between q' & q''
class SimpleLocalPlanner2D:
  def __init__(self, Q_space, resolution=0.01):
    self.configuration_space = Q_space
    self.resolution = Decimal(resolution)

  def increment_to_goal(self, q_start, q_goal):
    v_start, v_goal = Vector2D.from_configuration2D(q_start), \
                      Vector2D.from_configuration2D(q_goal)
    d_vector = Vector2D.displacement_vector(v_start, v_goal)
    d_hat = d_vector.unit_vector()
    increment_vector = d_hat * self.resolution

    return Configuration2D.from_vector2D(v_start + increment_vector)

  def find_path(self, q_start, q_goal):
    self.path = []
    # find path or return None
    if not self.configuration_space.is_collision_free(q_start) and \
       not self.configuration_space.is_collision_free(q_goal):
      return None

    self.path.append(q_start)

    # check for collision in increments of resolution until q_goal
    # TODO: optimise this algorithm, lazy b-tree or array mapping (parallel)
    current_config = self.increment_to_goal(q_start, q_goal)
    while current_config.x - q_goal.x >= self.resolution or \
          current_config.y - q_goal.y >= self.resolution:

      # check for collision
      if not self.configuration_space.is_collision_free(current_config):
        return None

      # add config to path
      self.path.append(current_config)

    self.path.append(q_goal)

    return self.path


class ProbabilisticRoadmap:
  def __init__(self, num_samples: int, num_nearest_neighbours: int,
               Q: ConfigurationSpace2D, local_planner: SimpleLocalPlanner2D = None):
    self.configuration_space = Q
    self.num_samples = num_samples
    self.num_nearest_neighbours = num_nearest_neighbours
    self.local_planner = local_planner

    # initialise roadmap graph, G = (V,E)
    self.nodes = []
    self.edges = []

  def build_roadmap(self):
    """
    Constructs the roadmap by sampling collision-free configurations
    and connecting them using local planner.

    Returns:
      None
    """

    pass

  def sample_configuration(self):
    """
    Samples a collision-free configuration within the configuration space.

    Returns:
      Configuration2D: The sampled configuration.
    """

    return sample_uniform_configuration2D(self.configuration_space.bounds)

  def is_collision_free(self, configuration):
    """
    Checks if the given configuration is collision-free within the configuration space.

    Args:
      configuration (Configuration2D): The configuration to check for collisions.

    Returns:
      bool: True if the configuration is collision-free, False otherwise.
    """

    return self.configuration_space.is_collision_free(configuration)

  def get_nearest_neighbours(self, node: Node) -> List[Node]:
    """
    Finds the k-nearest neighbors of the given node within the roadmap.

    Args:
      node (Node): The node for which to find nearest neighbors.

    Returns:
      List[Node]: A list of k nearest neighbor nodes.
    """
    pass

  def local_path_exists(self, q_start, q_goal):
    """
    Checks whether a collision-free path exists between the start and end configurations.

    Args:
      start_configuration (Configuration2D): The start configuration.
      end_configuration (Configuration2D): The end configuration.

    Returns:
      bool: True if a collision-free path exists, False otherwise.
    """

    if self.local_planner is None:
      self.local_planner = SimpleLocalPlanner2D(self.configuration_space)

    # Find nodes corresponding to q_start and q_goal
    start_node, end_node = None, None
    for node in self.nodes:
      if node.configuration == q_start:
        start_node = node
      elif node.configuration == q_goal:
        end_node = node

    if start_node and end_node:
      local_path = self.local_planner.find_path(q_start, q_goal)

    if local_path:
      self.add_edge(start_node, end_node)  # TODO: keep here or build_roadmap?
      return True
    else:
      return False

  def add_node(self, configuration):

    self.nodes.append(Node(configuration))

  def add_edge(self, start_node, end_node):

    self.edges.append((start_node, end_node))



# helper
import random
import decimal
from typing import Tuple


def sample_uniform_configuration2D(bounds: RectangularBounds2D,
                                   resolution=0.01) -> Tuple[float, float]:
  """
    Sample a 2D uniform configuration within the given bounds.

    Args:
      bounds (Tuple[Tuple[float, float], Tuple[float, float]]):
        The bounds () within which to sample the 2D configuration.

    Returns:
      Tuple[float, float]:
        A tuple representing the sampled 2D configuration in the form (x, y).

    Sample code:
      bounds = RectangularBounds2D((0, 10), (0, 10))
      sampled_config = sample_uniform_configuration2D(bounds)
      print(sampled_config)
  """

  # Sample x coordinate
  x_value = random.uniform(bounds.x_min, bounds.x_max)
  # Round the x value to two decimal places
  x_value = decimal.Decimal(x_value).quantize(decimal.Decimal("0.01"))
  # Sample y coordinate
  y_value = random.uniform(bounds.y_min, bounds.y_max)
  # Round the y value to two decimal places
  y_value = decimal.Decimal(y_value).quantize(decimal.Decimal("0.01"))
  # Return (x, y) coordinates as floats
  return (float(x_value), float(y_value))

def get_nearest_neighbours2D(node: Node,
                             num_nearest_neighbours: int,
                             node_list: List[Node],
                             distance_metric = None) -> List[Node]:
  """
  Finds the k-nearest neighbors of the given node within the roadmap.

  Args:
    node (Node): The node for which to find nearest neighbors.
    num_nearest_neighbours (int): The number, k, of nearest nodes to return.
    node_list (List): List of nodes to search.
    distance_metric (Function): Function to calculate distance between nodes.

  Returns:
    List[Node]: A list of k nearest neighbor nodes.
    """
  pass


In [ ]:
# Algorithm 2: query roadmap algorithm (pseudocode)

# Inputs:
# qinit: the initial configuration
# qgoal: the goal configuration
# k: the number of closest neighbours to examine for each configuration
# G = (V,E): the roadmap computed by algorithm 1
#
# Output:
# A path from qinit to qgoal or "failure"

# qinit_neighbourhood = get_neighbours(qinit, k, distance_metric, V)
# qgoal_neighbourhood = get_neighbours(qgoal, k, distance_metric, V)
# V.extend({qinit, qgoal})
# qprime = get_closest_neighbours(qinit, qinit_neighbourhood)  # NOTE: return ranked list of neighbours (same as get_neighbours)
#
# for close_neighbour in qprime:
#   if local_path_exists(qinit, close_neighbour, local_planner):
#     E.append((qinit, close_neighbour))
#     break
#
# q_prime = get_closest_neighbours(qgoal, qgoal_neighbourhood)
# for close_neighbour in q_prime:
#   if local_path_exists(qgoal, close_neighbour, local_planner):
#     E.append((qinit, close_neighbour))
#     break
#
# P = find_shortest_path(start=qinit, end=qgoal, on=G)
# if P is not NULL:
#   return P
# else:
#   return 'failure'